# Foodie Recommender Data Model (V0.1)
We'll choose a set of starter features and attempt to train a two-towered recommender system

## Data Models

In [30]:
import dateutil
from pydantic import BaseModel, validator
import os
from typing import Any, List

import json


genres = {}



def save_model_data(model: BaseModel, filename: str) -> str:
    with open(filename, 'w+') as file:
        json.dump(model.json(), file)
    return filename



def create_empty_genres_file(filepath: str) -> None:
    with open(filepath, 'w+') as file:
        json.dump({}, file)
    return {}
        
        

def load_or_create_genres(genres_file='genres.json') -> dict:
    if os.path.isfile(genres_file):
        with open(genres_file, 'r') as file:
            return json.load(file)
    else:
        print('file not found', genres_file)
        return create_empty_genres_file(genres_file)


    
def save_genres(genres: dict, genres_file='genres.json') -> dict:
    with open(genres_file, 'w+') as file:
        json.dump(genres, file)
    return genres

    
        
def add_element_to_genres(element: str) -> int:
    genres = load_or_create_genres()
    dict_len = len(genres)
    if element not in genres:
        genres[element] = dict_len + 1
        genres = save_genres(genres)
    return genres[element]
        
        
        
def map_genre(genre_list: list) -> list:
    tmp = []
    for element in genre_list:
        tmp.append(add_element_to_genres(element))
    return tmp
        

    
class RestaurantUser(BaseModel):
    user_birth_date: int
    user_genres: List[Any] = [0]
    user_id: int
    user_occupation: str
    user_gender: bool  # 0: male, 1: female
    user_zip_code: int
    
    
    @validator('user_genres')
    def index_or_add(cls, v):
        assert len(v) > 0, 'Must provide list of genre > 0'
        return map_genre(v)
    
    def save(self, prefix='user') -> None:
        name = prefix + f"_{self.user_id}.json"
        return save_model_data(self, name)

    
        
class Restaurant(BaseModel):
    restaurant_id: int
    restaurant_title: str
    restaurant_genres: List[Any]
    restaurant_zip_code: int
   
    @validator('restaurant_genres')
    def index_or_add(cls, v):
        assert len(v) > 0, 'Must provide list of genre > 0'
        return map_genre(v)
        
    def save(self, prefix='restaurant') -> str:
        name = prefix + f"_{self.restaurant_id}.json"
        return save_model_data(self, name)
    
    
    
class RestaurantRating(BaseModel):
    user: RestaurantUser
    restaurant: Restaurant
    timestamp: int  # Converting all date/time to posix integer
    restaurant_rating: int
    
    @validator('timestamp')
    def convert_valid_time(cls, v: str):
        pass
    
    def save(self, prefix='rating') -> str:
        name = prefix + f"_{self.restaurant}_{self.user}_{self.timestamp}.json"
        return save_model_data(self, name)
    
    def flatten(self) -> dict:
        tmp = {
            **self.user.dict(), **self.restaurant.dict(),
            'timestamp': self.timestamp,
            'restaurant_rating': self.restaurant_rating,
        }
        return tmp

    
    

In [31]:
# Create a test user
user_profile = {
    "user_birth_date": 20220101,
    "user_genres": ['vegetarian', 'thai'],
    "user_id": 1001,
    "user_occupation": "student",
    "user_gender": 0,  # 0: male, 1: female
    "user_zip_code": 84104,
}

user = RestaurantUser(**user_profile)

# Create a test restaurant
restaurant_profile = {
    "restaurant_id": 1,
    "restaurant_title": "skinnyfats",
    "restaurant_genres": ['vegetarian', 'thai', 'healthy', 'fried'],
    "restaurant_zip_code": 84104,
}

restaurant = Restaurant(**restaurant_profile)

# Create a test rating
rating_profile = {
    "user": user,
    "restaurant": restaurant,
    "timestamp": 202201012200,  # Converting all date/time to posix integer
    "restaurant_rating": 10,
}

rating = RestaurantRating(**rating_profile)

In [35]:
# The rating object contains user and restaurant models 
# Here, we implement a helper fn flatten to make it an easier document to deal with
rating.flatten()

{'user_birth_date': 20220101,
 'user_genres': [1, 2],
 'user_id': 1001,
 'user_occupation': 'student',
 'user_gender': False,
 'user_zip_code': 84104,
 'restaurant_id': 1,
 'restaurant_title': 'skinnyfats',
 'restaurant_genres': [1, 2, 3, 4],
 'restaurant_zip_code': 84104,
 'timestamp': None,
 'restaurant_rating': 10}

In [37]:
# Our user model can output a dictionary as well with a direct call to the dict() method
user.dict()

{'user_birth_date': 20220101,
 'user_genres': [1, 2],
 'user_id': 1001,
 'user_occupation': 'student',
 'user_gender': False,
 'user_zip_code': 84104}

## Convert to Tensorflow Dataset
We'll now arbitrarily copy the data and create Tensorflow datasets to train new embeddigns models with